In [1]:
import os
import torch
import albumentations as A
from torchvision import models
from torch.utils.data import DataLoader
from albumentations.pytorch import ToTensorV2
from glob import glob

from required.dataset_temp import CustomDataset
from required.utils import save_model

In [2]:
def test(model, test_loader, device):
    model.eval()

    correct = 0
    total = 0
    with torch.no_grad():
        for batch_index, (data, target) in enumerate(test_loader):
            data, target = data.to(device), target.to(device)  # target은 정답(라벨)
            output = model(data)                               # output은 모델이 예측한 값 
            _, argmax = torch.max(output, 1)
            
            total += target.size(0)  # 전체 사이즈
            correct += (target == argmax).sum().item()  # target과 output 비교
        
        acc = correct / total * 100
        print("acc for {} image : {:.2f}%".format(total, acc))

        return acc  

In [3]:

def main():
    test_transform = A.Compose([
        A.Resize(height=256, width=256),
        ToTensorV2()
    ])    

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = models.__dict__["resnet18"](pretrained=False, num_classes = 5)  # resmodel18 모델, num_classes 중요 !!!
    model = model.to(device)

    model_pt_paths = glob(os.path.join("./model_save", "*", "final.pt"))

    best_acc = 0  # accuracy of the best model (제일 처음에는 0으로 초기화)
    best_trans_name = "None"  # 제일 처음 초기화

    for pt_path in model_pt_paths:
        transform_name = pt_path.split("\\")[1]  # transform0
        
        model.load_state_dict(torch.load(pt_path, map_location= device))
        test_dataset = CustomDataset("./data/val", transform= test_transform)
        test_loader = DataLoader(test_dataset, batch_size= 1, shuffle= False)
        
        print(f"===== The Result of {transform_name} =====\n")
        new_acc = test(model, test_loader, device)

        if new_acc > best_acc:
            print(f"{transform_name}이 {best_trans_name}보다 정확도가 높습니다\n")
            save_model(
                model= model,
                save_dir= "./model_save",
                file_name= f"best.pt"
            )
            best_acc = new_acc
            best_trans_name = transform_name
        elif new_acc == best_acc:
            print(f"{transform_name}이 {best_trans_name}보다 정확도가 같습니다\n")
        else:
            print(f"{best_trans_name}의 정확도가 더 높습니다\n")
        
if __name__ == '__main__':
    main()

c:\Users\tempe\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tempe\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
